In [1]:
import numpy as np
import alexas_functions
import matplotlib.pyplot as plt
import xarray as xr
from sklearn.decomposition import PCA
import cartopy.crs as ccrs

nn34 = alexas_functions.nn34
ep = alexas_functions.ep
mc = alexas_functions.mc
alexas_functions.list_my_functions()
sea100_can= alexas_functions.get_landsea_mask(np.loadtxt('C:\\Users\\alexa\\Documents\\RESEARCH\\Alexa_Zabaske\\Python_Notebooks\\datafiles\\CanESM2_OceanPercents.txt'), mtype='sea')
sea100_mpi = alexas_functions.get_landsea_mask(np.loadtxt('C:\\Users\\alexa\\Documents\\RESEARCH\\Alexa_Zabaske\\Python_Notebooks\\datafiles\\MPI-ESM_OceanPercents.txt'), mtype='sea')

the available functions are: 
list_my_functions
test_function
get_filename
get_CMIP_name_list
set_new_time_variable
get_landsea_mask
extract_region
mask_out_regions
reshape
get_PC_components
cc_ev
dump_into_pickle
open_pickle_data
uniform_coords
zonal_avg
Fourier_Analysis
:end of list.


In [ ]:
##########  -----  INIT  -----  ##########
num_ensms=1
num_of_pcs=3

daatype = 'can_ensm'
name='CanESM2'
sea100_daa = sea100_can

# daatype = 'mpi_ensm'
# name='MPI-ESM'
# sea100_daa = sea100_mpi

## specify pca type (global? nh? other?)
## specify box type (fixed points)
## if going to add a loop, add before extracting region
## then correlate with each PRECIP pca
## save correlation values into a pandas data frame? 
## especially the fixed data, like nn34, ep-mc, etc. 

## to make a heat map, use a fixed box size, and shift it throughout different placements.
## calculate the correlation of the sst to the global precip at each box centroid
########## INIT ##########

###### OUTCOME ######
cc_nn34 = np.zeros((4, num_ensms, num_of_pcs))
ev_nn34 = np.zeros((4, num_ensms, num_of_pcs))

# cc_epmc = np.zeros((4, num_ensms, num_of_pcs))
# ev_epmc = np.zeros((4, num_ensms, num_of_pcs))


##### BEGIN LOOPING ENSEMBLE MEMBERS
for n_e in range(0, num_ensms):

    ##### ---- OPEN ts (SST) FILE for single ensemble member
    list_of_files_ts = []
    filename_ts = alexas_functions.get_filename(daatype, 'historical', 'ts', name, r=n_e)
    list_of_files_ts.append(filename_ts)
    
    if daatype=='mpi_ensm':
        list_of_files_ts.append( alexas_functions.get_filename(daatype, 'rcp85', 'ts', name, r=n_e) )
        
    ts_mon = xr.open_mfdataset(list_of_files_ts)
    ts_mon.close()
    
    ## apply mask to make ts only contain 100% ocean points
    ts_mon_sea100 = ts_mon.copy()
    ts_mon_sea100['ts'].values = ts_mon_sea100['ts'].values*sea100_daa

    ## sample the monthly sst to seasonal means
    ts_seas = ts_mon_sea100.resample(time="Q-NOV").mean() 
    ts_seas = ts_seas.isel(time=(slice(2,-3))) ## lines it up to JJA, SON, DJF, MAM

    ## extract nino3.4 region, calculating sst over reagion in return form function
    nn34_seas = alexas_functions.extract_region(ts_seas, nn34[0], nn34[1], nn34[2], nn34[3], mean='yes')


    ################################################################
    ##### ----  OPEN PRECIP FILE(s) for single ensemble member
    list_of_files_pr = []
    filename_pr = alexas_functions.get_filename(daatype, 'historical', 'pr', name, r=n_e)
    list_of_files_pr.append(filename_pr)
    
    if daatype=='mpi_ensm':
        list_of_files_pr.append( alexas_functions.get_filename(daatype, 'rcp85', 'pr', name, r=n_e) )    

    pr_mon = xr.open_mfdataset(list_of_files_pr)
    pr_mon.close()
    
    print(filename_pr)
    
    ## save lat and lon size data 
    latsize=len(pr_mon.lat.values)
    lonsize=len(pr_mon.lon.values)
    timesize_mon = len(pr_mon.time.values)
    #print(latsize, lonsize, timesize_mon)


    ## sample the seasonal pr data to seasonal means
    pr_seas = pr_mon.resample(time="Q-NOV").mean()
    pr_seas = pr_seas.isel(time=(slice(2,-3)))


    ## multiply the precip by the latitudinal cosine weights
    lat_cos_weights = np.cos(np.deg2rad(pr_seas.lat.values)).reshape((latsize,1)) 
    pr_seas.pr.values = pr_seas.pr.values *lat_cos_weights

    ## INITIALIZE PCA DATA LISTS
    
    ## -- TS: PC time series for all seasons 
    pr_PCTS_glob = []

    ## -- G: Grid spatial map of PCA for all seasons
    pr_PCG_glob = []

    ## -- expl: explained variance for all seasons
    pr_expl_var_glob = []
    
    ##### BEGIN LOOPING EACH SEASON
    season_nums = np.array([8,11,2,5]) #JJA, SON, DJF, MMA
    for s in range(4):

        seas=season_nums[s]
        pr_1seas = pr_seas.sel(time=pr_seas['time.month'] ==  seas)
        pr_glob = pr_1seas

        PCG_glob, PCTS_glob, expl_var_glob = get_PC_components(pr_glob, num_of_pcs, opt='all')
        pr_PCG_glob.append(PCG_glob)
        pr_PCTS_glob.append(PCTS_glob)
        pr_expl_var_glob.append(expl_var_glob)


        nn34_1seas = nn34_seas.sel( time = nn34_seas['time.month'] == seas )['ts']

        #epmc_1seas = epmc_seas.sel( time = epmc_seas['time.month'] == seas )['ts']

        for pci in range(num_of_pcs):

            cc_nn34[s, n_e-1, pci], ev_nn34[s, n_e-1, pci] = cc_ev(nn34_1seas, pr_PCTS_glob[s][pci])
            #cc_epmc[s, n_e-1, pci], ev_epmc[s, n_e-1, pci] = cc_ev(epmc_1seas, pr_PCTS_glob[s][pci])